# **ETL TEST**

## CONFIGURACIÓN INICIAL

In [1]:
# descargar librerías
!pip install reverse_geocoder
!pip install seaborn

In [2]:
# importar librerías
import pandas as pd
import datetime
import reverse_geocoder as rg
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [3]:
# funcion para pasar de fecha de nacimiento a edad
def from_dob_to_age(born):
    today = datetime.date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [4]:
# leer csv
df_perf_test = pd.read_csv(r'https://raw.githubusercontent.com/Ciarzi/DP3_GP1/main/datasets/test_performance.csv')
df_loan_test = pd.read_csv(r'https://raw.githubusercontent.com/Ciarzi/DP3_GP1/main/datasets/test_previous_loan.csv')
df_loan_train = pd.read_csv(r'https://raw.githubusercontent.com/Ciarzi/DP3_GP1/main/datasets/train_previous_loan.csv')
df_dem_test = pd.read_csv(r'https://raw.githubusercontent.com/Ciarzi/DP3_GP1/main/datasets/test_datos_demograficos.csv')
df_dem_train = pd.read_csv(r'https://raw.githubusercontent.com/Ciarzi/DP3_GP1/main/datasets/train_datos_demograficos.csv')
df_loc = pd.read_csv(r'https://raw.githubusercontent.com/Ciarzi/DP3_GP1/main/datasets/own_data/datos_localizacion.csv')

## ANEXAR DATOS DE CLIENTES Y DE PRÉSTAMOS ANTIGUOS

In [5]:
#adjuntar los dos csv
df_dem = df_dem_test.append(df_dem_train, ignore_index=True)
#eliminar filas que sean idénticas
df_dem.drop_duplicates(inplace = True)
#dimensiones del nuevo dataframe
df_dem.shape

C:\Users\sergi\AppData\Local\Temp/ipykernel_14064/1409091891.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_dem = df_dem_test.append(df_dem_train, ignore_index=True)


(5818, 9)

In [6]:
#adjuntar los dos csv
df_loan = df_loan_test.append(df_loan_train, ignore_index=True)
#eliminar filas que sean idénticas
df_loan.drop_duplicates(inplace = True)
#dimensiones del nuevo dataframe
df_loan.shape

C:\Users\sergi\AppData\Local\Temp/ipykernel_14064/3920205185.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_loan = df_loan_test.append(df_loan_train, ignore_index=True)


(24090, 12)

## PREPARAR DATOS PRÉSTAMOS ANTIGUOS

In [7]:
# vista previa
df_loan.head()

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,closeddate,referredby,firstduedate,firstrepaiddate
0,8a858899538ddb8e015390510b321f08,301621635,3,2016-05-17 10:37:00.000000,2016-05-17 09:36:55.000000,10000.0,13000.0,30,2016-06-17 00:04:15.000000,NaN,2016-06-16 00:00:00.000000,2016-06-16 15:44:08.000000
1,8a858959537a097401537a4e316e25f7,301810201,5,2017-02-04 21:28:59.000000,2017-02-04 20:28:52.000000,30000.0,36800.0,60,2017-03-02 16:22:58.000000,NaN,2017-03-06 00:00:00.000000,2017-03-02 16:07:47.000000
2,8a858959537a097401537a4e316e25f7,301831255,6,2017-03-04 10:28:22.000000,2017-03-04 09:28:16.000000,30000.0,34400.0,30,2017-04-02 00:44:24.000000,NaN,2017-04-03 00:00:00.000000,2017-04-01 21:29:46.000000
3,8a8589c253ace09b0153af6ba58f1f31,301627292,3,2016-06-02 14:27:14.000000,2016-06-02 13:27:08.000000,10000.0,13000.0,30,2016-07-04 11:34:04.000000,NaN,2016-07-04 00:00:00.000000,2016-07-04 11:19:01.000000
4,8a8589c253ace09b0153af6ba58f1f31,301621095,2,2016-05-16 09:13:12.000000,2016-05-16 08:13:04.000000,10000.0,11500.0,15,2016-06-02 00:02:58.000000,NaN,2016-05-31 00:00:00.000000,2016-06-01 15:22:34.000000


In [8]:
#pasar columnas a fecha
df_loan['firstduedate'] = pd.to_datetime(df_loan['firstduedate'])
df_loan['firstrepaiddate'] = pd.to_datetime(df_loan['firstrepaiddate'])
df_loan['firstrepaiddate'] = pd.to_datetime(df_loan['firstrepaiddate'].dt.date)

In [9]:
# guardar si se ha hecho el primer pago antes o después del plazo
df_loan['pago_plazo'] = df_loan.apply(lambda x: 1 if (x['firstduedate'] >= x['firstrepaiddate']) else -3, axis=1)

In [10]:
#nos quedamos con las columnas que importan
df_loan = df_loan[['customerid','pago_plazo']]

In [11]:
#agrupa el dataframe por cliente, obtienendo para cada uno si ha pagado en plazo o no sus prestamos
df_loan_gr = df_loan.groupby(['customerid']).sum()
df_loan_gr.head()

,pago_plazo
customerid,
8a1088a0484472eb01484669e3ce4e0b,-3
8a1a1e7e4f707f8b014f797718316cad,0
8a1a32fc49b632520149c3b8fdf85139,3
8a1eb5ba49a682300149c3c068b806c7,4
8a1edbf14734127f0147356fdb1b1eb2,2


In [12]:
# pasar la columna a binario
df_loan_gr['pagos_plazo'] = 1
df_loan_gr.loc[df_loan_gr['pago_plazo'] < 0, 'pagos_plazo'] = 0
df_loan_gr = df_loan_gr['pagos_plazo']

## ADJUNTAR TODOS LOS DATOS EN UN SOLO DATAFRAME

In [13]:
df_perf_test.shape

(1450, 9)

Añadir información de los clientes

In [14]:
# se añade la información del cliente
df_test = df_perf_test.merge(df_dem, on='customerid', how='left')
#ver tamaño del nuvo df
df_test.shape

(1450, 17)

Añadir localización de los clientes, que se obtiene de un csv diferente, el cual se ha obtenido con el proceso 1_etl_localizacion.ipynb ya que es un proceso que ha tomado 3 horas, por lo que se ha guardado los resultados en un archivo para tener que ejecutarlo una única vez.

In [15]:
# se añade la localización de los clientes
df_test = df_test.merge(df_loc, on='customerid', how='left')
#ver tamaño del nuvo df
df_test.shape

(1450, 19)

Añadir información de los préstamos antiguos de los clientes

In [16]:
# se añade la información de préstamos antiguos del cliente
df_test = df_test.merge(df_loan_gr, on='customerid', how='left')
#ver tamaño del nuvo df
df_test.shape

(1450, 20)

Añadir información de los préstamos antiguos de los referidos

In [17]:
# se añade la información de préstamos antiguos del referido del cliente
df_test = df_test.merge(df_loan_gr, left_on='referredby', right_on='customerid', how='left')
#ver tamaño del nuvo df
df_test.shape

(1450, 21)

In [18]:
df_test.head()

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,referredby,birthdate,...,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients,pais,estado,pagos_plazo_x,pagos_plazo_y
0,8a858899538ddb8e015390510b321f08,301998974,4,40:48.0,39:35.0,10000,12250.0,30,NaN,1983-12-16 00:00:00.000000,...,5.835219,8.274730,First Bank,NaN,Permanent,NaN,NG,Kogi,0.0,NaN
1,8a858959537a097401537a4e316e25f7,301963615,10,43:40.0,42:34.0,40000,44000.0,30,NaN,1986-04-06 00:00:00.000000,...,3.245768,6.601778,First Bank,NaN,Permanent,NaN,NG,Lagos,1.0,NaN
2,8a8589c253ace09b0153af6ba58f1f31,301982236,6,15:11.0,15:04.0,20000,24500.0,30,NaN,1981-09-05 00:00:00.000000,...,3.227945,6.586668,UBA,NaN,Permanent,NaN,NG,Lagos,0.0,NaN
3,8a858e095aae82b7015aae86ca1e030b,301971730,8,00:54.0,00:49.0,30000,34500.0,30,NaN,1977-11-16 00:00:00.000000,...,6.106486,7.287064,Zenith Bank,NaN,NaN,NaN,NG,Edo,1.0,NaN
4,8a858e225a28c713015a30db5c48383d,301959177,4,04:33.0,04:27.0,20000,24500.0,30,NaN,1975-08-25 00:00:00.000000,...,5.248368,13.059864,UBA,NaN,Permanent,NaN,NG,Sokoto,1.0,NaN


## TRANSFORMACIONES INICIALES EN EL DATASET

In [20]:
# quitar columnas que no se usan
# elegir las columnas
df_test_delete = ['level_of_education_clients', 'customerid', 'systemloanid','bank_name_clients', 'bank_branch_clients', 'creationdate', 'longitude_gps', 'latitude_gps']
# borrar las columnas elegidas
df_test.drop(columns = df_test_delete, inplace = True)

In [21]:
#ver tipo de datos de las columnas
df_test.dtypes

loannumber                     int64
approveddate                  object
loanamount                     int64
totaldue                     float64
termdays                       int64
referredby                    object
birthdate                     object
bank_account_type             object
employment_status_clients     object
pais                          object
estado                        object
pagos_plazo_x                float64
pagos_plazo_y                float64
dtype: object

In [22]:
# la columna de fecha de aprovación está en un formato extraño
df_test['approveddate']

0       40:48.0
1       43:40.0
2       15:11.0
3       00:54.0
4       04:33.0
         ...   
1445    47:02.0
1446    00:45.0
1447    09:40.0
1448    47:54.0
1449    12:09.0
Name: approveddate, Length: 1450, dtype: object

In [23]:
# por tanto, no se puede usar
df_test.drop(columns = ['approveddate'], inplace = True)

In [24]:
# cambiar el tipo de dato de algunas columnas
#cambiar tipo de dato
df_test['totaldue'] = df_test['totaldue'].astype('int')
df_test['pagos_plazo_x'] = df_test['pagos_plazo_x'].astype('Int64')
df_test['pagos_plazo_y'] = df_test['pagos_plazo_x'].astype('Int64')
#pasar columna a fecha
df_test['birthdate'] = pd.to_datetime(df_test['birthdate'])

In [25]:
# ver registros nulos en las columnas
print(df_test.isnull().sum())

loannumber                      0
loanamount                      0
totaldue                        0
termdays                        0
referredby                   1266
birthdate                       0
bank_account_type               0
employment_status_clients     208
pais                            0
estado                          0
pagos_plazo_x                   8
pagos_plazo_y                   8
dtype: int64


In [26]:
# aparecen 8 registros donde los clientes no tienen datos de préstamos antiguos
# sin embargo, la columna loannumber indica lo contrario
df_test['loannumber'].min()

2

In [27]:
# al ser tan pocos registros, se opta por borrarlos
df_test.dropna(subset=['pagos_plazo_x'], inplace = True)

In [28]:
df_test.head()

,loannumber,loanamount,totaldue,termdays,referredby,birthdate,bank_account_type,employment_status_clients,pais,estado,pagos_plazo_x,pagos_plazo_y
0,4,10000,12250,30,NaN,1983-12-16,Other,Permanent,NG,Kogi,0,0
1,10,40000,44000,30,NaN,1986-04-06,Savings,Permanent,NG,Lagos,1,1
2,6,20000,24500,30,NaN,1981-09-05,Savings,Permanent,NG,Lagos,0,0
3,8,30000,34500,30,NaN,1977-11-16,Savings,NaN,NG,Edo,1,1
4,4,20000,24500,30,NaN,1975-08-25,Savings,Permanent,NG,Sokoto,1,1


## TRANSFORMACIONES EN COLUMNAS

### Histórico de pagos del cliente

- 0: El cliente tiene un mal historial de pagos
- 1: El cliente tiene un buen historial de pagos

In [29]:
df_test.rename(columns={'pagos_plazo_x': 'historial'}, inplace = True)


### Referido

- 0: El cliente ha sido referido por un cliente con un mal historial de pagos
- 1: El cliente no ha sido referido
- 2: El cliente ha sido referido por un cliente sin historial de pagos
- 3: El cliente ha sido referido por un cliente con un buen historial de pagos

In [30]:
df_test['referido'] = 0
df_test.loc[df_test['referredby'].isnull(), 'referido'] = 1
df_test.loc[(df_test['referredby'].notnull()) & (df_test['pagos_plazo_y'].isnull()), 'referido'] = 2
df_test.loc[(df_test['referredby'].notnull()) & (df_test['pagos_plazo_y'] == 1), 'referido'] = 3
# borrar columnas que ya no se usan
df_test.drop(columns = ['referredby', 'pagos_plazo_y'], inplace = True)

### Edad

In [31]:
# pasar de fecha de nacimiento a edad
df_test['age'] = df_test['birthdate'].apply(lambda x: from_dob_to_age(x)).astype('Int64')
# quitar la columna que ya no se usa
df_test.drop(columns = ['birthdate'], inplace = True)

### Deuda por día

In [32]:
df_test['due_per_day'] = df_test.apply(lambda x: x['totaldue'] / x['termdays'], axis=1)
df_test['due_per_day'] = df_test['due_per_day'].astype('int')

### Interés

In [33]:
df_test['interes'] = df_test.apply(lambda x: ((x['totaldue'] - x['loanamount']) / x['loanamount']) * 100, axis=1)
# borrar columnas que ya no se usan
df_test.drop(columns = ['termdays', 'totaldue', 'loanamount'], inplace = True)

### Tipo de cuenta del banco

Dummies para cada tipo

In [34]:
#ver distribución
df_test['bank_account_type'].value_counts(dropna = False)

Savings    1146
Other       280
Current      16
Name: bank_account_type, dtype: int64

In [35]:
# one hot encoding
df_test = pd.concat([df_test,pd.get_dummies(df_test['bank_account_type'], prefix='bank_account_type')],join='inner',axis=1)
df_test.drop(['bank_account_type'],axis=1, inplace=True)
df_test.head()

,loannumber,employment_status_clients,pais,estado,historial,referido,age,due_per_day,interes,bank_account_type_Current,bank_account_type_Other,bank_account_type_Savings
0,4,Permanent,NG,Kogi,0,1,38,408,22.5,0,1,0
1,10,Permanent,NG,Lagos,1,1,36,1466,10.0,0,0,1
2,6,Permanent,NG,Lagos,0,1,40,816,22.5,0,0,1
3,8,NaN,NG,Edo,1,1,44,1150,15.0,0,0,1
4,4,Permanent,NG,Sokoto,1,1,46,816,22.5,0,0,1


In [36]:
# Cambiar nombre de las columnas
cambios_nombre = {'bank_account_type_Current':'cuenta_corriente', 'bank_account_type_Other':'cuenta_otra', 'bank_account_type_Savings':'cuenta_ahorro'}
df_test.rename(columns = cambios_nombre, inplace = True)

### Localización

In [37]:
# vemos de que pais hay información
df_test['pais'].value_counts(dropna = False)

NG    1434
US       2
CN       1
AR       1
GH       1
AE       1
GB       1
BJ       1
Name: pais, dtype: int64

Más del 99% de los datos son de Nigeria. Para estos casos, se va añadir el pib per cápita del estado de Nigeria, y para los casos fuera del país, se va a usar el pib per cápita de dicho país.

In [38]:
# leemos los csv con los pib per capita de los estados de Nigeria y de los otros países que aparecen
df_pib_nigeria = pd.read_csv(r'https://raw.githubusercontent.com/Ciarzi/DP3_GP1/main/datasets/own_data/nigeria_pib_capita.csv')
df_pib_mundo = pd.read_csv(r'https://raw.githubusercontent.com/Ciarzi/DP3_GP1/main/datasets/own_data/mundo_pib_capita.csv')

In [39]:
df_pib_nigeria.head()

,estado,pib_capita_estado
0,Abuja Federal Capital Territory,US$2317
1,Lagos,US$2174
2,Rivers,US$2396
3,Delta,US$2452
4,Oyo,US$1718


In [40]:
# transformar la columna de pib per capita de nigeria
# sacar el numero
df_pib_nigeria['pib_capita_estado'] = df_pib_nigeria['pib_capita_estado'].apply(lambda x: x[3:])
# pasar a tipo numero
df_pib_nigeria['pib_capita_estado'] = df_pib_nigeria['pib_capita_estado'].astype('int')
# pasamos de dolar a euro
df_pib_nigeria['pib_capita_estado'] = df_pib_nigeria['pib_capita_estado'] * 0.95
# redondear a euro
df_pib_nigeria['pib_capita_estado'] = df_pib_nigeria['pib_capita_estado'].astype('int')

In [41]:
df_pib_mundo.head()

,pais,pib_capita_pais
0,NG,0
1,US,63543
2,GB,40284
3,AE,43103
4,BJ,1291


In [42]:
# transformar la columna de pib per capita de los países
# pasamos de dolar a euro
df_pib_mundo['pib_capita_pais'] = df_pib_mundo['pib_capita_pais'] * 0.95
# redondear a entero
df_pib_mundo['pib_capita_pais'] = df_pib_mundo['pib_capita_pais'].astype('int')

In [43]:
# se añade el pib per capita de los otros países distintos a nigeria
df_test = df_test.merge(df_pib_mundo, on='pais', how='left')
#ver tamaño del nuvo df
df_test.shape

(1442, 13)

In [44]:
# se añade el pib per capita de los datos de Nigeria
df_test = df_test.merge(df_pib_nigeria, on='estado', how='left')
#ver tamaño del nuvo df
df_test.shape

(1442, 14)

In [45]:
# se crea la columna final de pib per cápita
df_test['pib_capita'] = df_test['pib_capita_pais']
df_test.loc[df_test['pib_capita'] == 0, 'pib_capita'] = df_test['pib_capita_estado']

In [46]:
#quitar columnas que ya no se usan
df_test.drop(columns = ['pais', 'estado', 'pib_capita_pais', 'pib_capita_estado'], inplace = True)

### Ocupación

- 0: Low Income: Retired, Unemployed, Student
- 1: NsNc  
- 2: High income: Permanent, Self-Employed, Contract

In [47]:
## transform a value inside a column to a separate boolean column
df_test['employment'] = 1
df_test.loc[df_test['employment_status_clients'] == 'Retired', 'employment'] = 0
df_test.loc[df_test['employment_status_clients'] == 'Unemployed', 'employment'] = 0
df_test.loc[df_test['employment_status_clients'] == 'Student', 'employment'] = 0
df_test.loc[df_test['employment_status_clients'] == 'Permanent', 'employment'] = 2
df_test.loc[df_test['employment_status_clients'] == 'Self-Employed', 'employment'] = 2
df_test.loc[df_test['employment_status_clients'] == 'Contract', 'employment'] = 2
#convertimos la columna a int
df_test['employment'] = df_test['employment'].astype('Int64')
# borrar columnas que ya no se usa
df_test.drop(columns = ['employment_status_clients'], inplace = True)

## GUARDAR CSV

In [49]:
# df a csv
df_test.to_csv(r'../datasets/own_data/test.csv', index = False)